In [ ]:
import json
with open("./word2abstract.json","r",encoding='utf-8') as f:
        word2abstract = json.load(f)

In [ ]:
with open("./300_path_2.json","r",encoding='utf-8') as f: # key:word value: its corresponding image files dir
    word2path = json.load(f)

In [ ]:
words = []
for key in word2abstract.keys():
    words.append(key)

In [ ]:
#from SPARQLWrapper import SPARQLWrapper, JSON
import multiprocessing
from multiprocessing.dummy import Pool as ThreadPool
from functools import partial
import time
import json

import subprocess
from subprocess import PIPE, Popen
import os
from tqdm import tqdm
import re
import requests

process_num = 3
wrong = []

tic1 = time.time()
def worker(entityname, i):
    try:
        if i % 1000 == 0:
            print("{}:{}".format(i, entityname))
        
        params = {'entityname': entityname}  # parameters passed to the server
        params['text'] = word2abstract[entityname][:3000].replace('\\','').replace('(','').replace(')','').replace('-','') 
        params['path'] = word2path[entityname]
            
        if i%2 == 0: # multiple servers in case crashing
            url = "http://172.20.137.91:8045/test" # url for your server
        else:
            url = "http://172.20.137.91:8044/test"

        try:
            resp = requests.get(url,params=params)
            result = json.loads(resp.text)
        except Exception as e:
            print('{}',params)
        return entityname, result, i
    except Exception as e:
        print('error:', e)
        return entityname, i

def collectMyResult(result):
    try:
        word2clip[result[0]] = result[1]['results']
        word2files[result[0]] = result[1]['files']
        if(int(result[2]) %5000 == 0):
            global tic1
            print("Got result {}".format(result))
            toc1 = time.time()
            shijian = toc1-tic
            print("耗时:",shijian)
            tic1 = toc1
    except Exception as e:
        wrong.append(result[0])
        print('error:', e)

if __name__ == "__main__":
    tic = time.time()
    pool = multiprocessing.Pool(processes=process_num,maxtasksperchild=1)
    word2clip = {} # clip score
    word2files = {} # corresponding images

    featureClass = [[words[i], i] for i in tqdm(range(0, len(words),1))] #list of arguments
    for f in featureClass:
        abortable_func = partial(worker)
        pool.apply_async(abortable_func, args=f,callback=collectMyResult)
    pool.close()
    pool.join()
                                           
    toc = time.time()
    shijian = toc-tic
    print("time consuming:",shijian)

    with open("./word2clip.json","w",encoding='utf-8') as f:
        json.dump(word2clip,f)
    
    with open("./word2files.json","w",encoding='utf-8') as f:
        json.dump(word2files,f)